### Este es el cuarto modelo. IDÉNTICO AL PRIMERO PERO AÑADIENDO ENSAMBLER. Incluye:
#### - Malla de parámetros.
#### - Datos de "Primer procesado de datos": sin variables categóricas y nan rellenos con media/moda.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# Exportar modelos
import pickle

# Pipeline
from sklearn.pipeline import Pipeline

# CV
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor

# Métricas
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error

In [2]:
# Cargamos datos
data = pd.read_csv('../data/processed/primer_procesado_entrenamiento.csv', sep = '\t')
target = pd.read_csv('../data/processed/primer_procesado_target.csv', sep = '\t')

In [3]:
# Cambiamos los nombres de las features para que no contengan espacios ni caracteres especiales
data.columns = ['pressure', 'mass_flux', 'x_e_out', 'D_e', 'D_h', 'length', 'chf_exp']


In [4]:
# Dividimos en features y target, siendo el target x_e_out [-]
X = data.drop(['x_e_out'], axis = 1)
y = data['x_e_out']
# Luego tendremos que recuperar los nombres originales de las features

In [5]:
# Dividimos entre train y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)


In [6]:
# Creamos un primer pie cualquier clasificador. Irá cambiando según va probando pero necesita 1.
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('estimator', Ridge())]
                )

In [7]:
# Creamos una malla de parámetros para cada modelo a probar

param_grid_linear_regression = {
    'estimator' : [LinearRegression()],
    'estimator__fit_intercept': [True, False],  # Probar con o sin intercepto
    'estimator__normalize': [True, False]  # Probar con o sin normalización de características
}

param_grid_ridge = {
    'estimator' : [Ridge()],
    'estimator__alpha': [0.1, 1.0, 10.0],  # Valores de alpha a probar
    'estimator__solver': ['svd', 'cholesky', 'lsqr']  # Métodos de solución a probar
}

param_grid_lasso = {
    'estimator' : [Lasso()],
    'estimator__alpha': [0.1, 1.0, 10.0],  # Valores de alpha a probar
    'estimator__max_iter': [1000, 2000, 5000]  # Número máximo de iteraciones a probar
}

param_grid_elasticnet = {
    'estimator' : [ElasticNet()],
    'estimator__alpha': [0.1, 1.0, 10.0],  # Valores de alpha a probar
    'estimator__l1_ratio': [0.25, 0.5, 0.75],  # Valores de l1_ratio a probar
    'estimator__max_iter': [1000, 2000, 5000]  # Número máximo de iteraciones a probar
}

param_grid_svr = {
    'estimator' : [SVR()],
    'estimator__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernels a probar
    'estimator__C': [0.1, 1.0, 10.0],  # Valores de C a probar
    'estimator__epsilon': [0.01, 0.1, 1.0]  # Valores de epsilon a probar
}

param_grid_decision_tree = {
    'estimator' : [DecisionTreeRegressor()],
    'estimator__max_depth': [None, 5, 10, 20],  # Profundidad máxima del árbol a probar
    'estimator__min_samples_split': [2, 5, 10],  # Número mínimo de muestras requeridas para dividir un nodo interno
    'estimator__min_samples_leaf': [1, 2, 4]  # Número mínimo de muestras requeridas en un nodo hoja
}

param_grid_random_forest = {
    'estimator' : [RandomForestRegressor()],
    'estimator__n_estimators': [10, 50, 100],  # Número de árboles a probar
    'estimator__max_depth': [None, 5, 10, 20],  # Profundidad máxima del árbol a probar
    'estimator__min_samples_split': [2, 5, 10],  # Número mínimo de muestras requeridas para dividir un nodo interno
    'estimator__min_samples_leaf': [1, 2, 4]  # Número mínimo de muestras requeridas en un nodo hoja
}

param_grid_gradient_boosting = {
    'estimator' : [GradientBoostingRegressor()],
    'estimator__n_estimators': [100, 200, 500],  # Número de estimadores a probar
    'estimator__learning_rate': [0.1, 0.01],  # Tasa de aprendizaje a probar
    'estimator__max_depth': [3, 5, 10],  # Profundidad máxima del árbol base a probar
    'estimator__subsample': [0.8, 1.0],  # Proporción de muestras utilizadas para entrenar cada árbol
}

param_grid_adaboost = {
    'estimator' : [AdaBoostRegressor()],
    'estimator__n_estimators': [50, 100, 200],  # Número de estimadores a probar
    'estimator__learning_rate': [0.1, 0.01, 0.005],  # Tasa de aprendizaje a probar
    'estimator__loss': ['linear', 'square', 'exponential']  # Función de pérdida a probar
}

param_grid_xgboost = {
    'estimator' : [XGBRegressor()],
    'estimator__n_estimators': [100, 200, 500],  # Número de árboles a probar
    'estimator__learning_rate': [0.1, 0.01, 0.001],  # Tasa de aprendizaje a probar
    'estimator__max_depth': [3, 5, 10],  # Profundidad máxima del árbol a probar
    'estimator__subsample': [0.8, 1.0],  # Proporción de muestras utilizadas para entrenar cada árbol
}

In [21]:
# Crear espacio de búsqueda 1
search_space = [
    param_grid_linear_regression,
    param_grid_ridge,
    param_grid_lasso,
    # param_grid_elasticnet,
    # param_grid_svr,
    param_grid_decision_tree,
    param_grid_random_forest,
    param_grid_gradient_boosting,
    # param_grid_adaboost,
    param_grid_xgboost # El mejor para el primer modelo
    ]

In [22]:
# Crear espacio de búsqueda 2
search_space2 = [
    # param_grid_linear_regression,
    # param_grid_ridge,
    # param_grid_lasso,
    param_grid_elasticnet,
    param_grid_svr, # Mejor luego quitarlo
    # param_grid_decision_tree,
    # param_grid_random_forest,
    # param_grid_gradient_boosting,
    param_grid_adaboost,
    # param_grid_xgboost # El mejor para el primer modelo
    ]

In [23]:
cv = KFold(n_splits=5, shuffle = True, random_state=0)

In [25]:
# Generar la cross-validation
gridsearch1 = GridSearchCV(
    estimator = pipe,  # Pipeline
    param_grid = search_space,  # Espacio de búsqueda
    cv = cv,  # Validación cruzada
    verbose=3,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)


In [26]:
# Generar la cross-validation
gridsearch2 = GridSearchCV(
    estimator = pipe,  # Pipeline
    param_grid = search_space2,  # Espacio de búsqueda
    cv = cv,  # Validación cruzada
    verbose=3,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

In [27]:
# Importamos VotingRegressor
from sklearn.ensemble import VotingRegressor


In [28]:
grid_model1 = gridsearch1
grid_model2 = gridsearch2

In [29]:
# Entrenamos la búsqueda
gridsearch1.fit(X_train, y_train)
gridsearch2.fit(X_train, y_train)

Fitting 5 folds for each of 256 candidates, totalling 1280 fits
Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('estimator', Ridge())]),
             n_jobs=-1,
             param_grid=[{'estimator': [ElasticNet()],
                          'estimator__alpha': [0.1, 1.0, 10.0],
                          'estimator__l1_ratio': [0.25, 0.5, 0.75],
                          'estimator__max_iter': [1000, 2000, 5000]},
                         {'estimator': [SVR(epsilon=0.01)],
                          'estimator__C': [0.1, 1.0, 10.0],
                          'estimator__epsilon': [0.01, 0.1, 1.0],
                          'estimator__kernel': ['linear', 'poly', 'rbf',
                                                'sigmoid']},
                         {'estimator': [AdaBoostRegressor()],
                          'estimator__learning_rate': [0.1, 0.01, 0.005],
                          'estimator__loss': ['linear', 'square'

In [21]:
# Obtener los mejores hiperparámetros y el mejor modelo
# best_params = gridsearch1.best_params_
# best_model = gridsearch2.best_estimator_
# print(best_params)
# print(best_model)


{'estimator': XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None,
             reg_alpha=None, reg_lambda=None, ...), 'estimator__learning_rate': 0.1, 'estimator__max_depth': 5, 'estimator__n_estimators': 200, 'estimator__subsample': 0.8}
Pipeline(steps=[('scaler', StandardScaler()),
                ('estimator',
                 XGBRegressor(base_score=0.5, booster='gbtree'

In [30]:
# Obtener los mejores modelos de GridSearch
best_model1 = grid_model1.best_estimator_
best_model2 = grid_model2.best_estimator_

In [32]:
# Crear el ensemble a partir de los mejores modelos
ensemble = VotingRegressor([('model1', best_model1), ('model2', best_model2)])

In [33]:
ensemble.fit(X_train, y_train)

VotingRegressor(estimators=[('model1',
                             Pipeline(steps=[('scaler', StandardScaler()),
                                             ('estimator',
                                              XGBRegressor(base_score=0.5,
                                                           booster='gbtree',
                                                           callbacks=None,
                                                           colsample_bylevel=1,
                                                           colsample_bynode=1,
                                                           colsample_bytree=1,
                                                           early_stopping_rounds=None,
                                                           enable_categorical=False,
                                                           eval_metric=None,
                                                           gamma=0, gpu_id=-1,
                                  

In [34]:
# Predecir los valores de salida para los datos de prueba utilizando el mejor modelo
y_pred = ensemble.predict(X_test)

In [36]:
# Evaluar el rendimiento del mejor modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Error cuadrático medio:', mse)
print('Raíz del error cuadrático medio:', np.sqrt(mse))
print('Error absoluto medio:', mean_absolute_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))
#print('Mejores hiperparámetros:', best_params)

Error cuadrático medio: 0.0060512554079105605
Raíz del error cuadrático medio: 0.07778981557961531
Error absoluto medio: 0.052410830791147694
R2: 0.43384654376451226


### Extraemos los mejores hiperparámetros

In [37]:
# Extraemos los mejores hiperparámetros de best_model
best_model_params = ensemble.get_params()
print(best_model_params)

{'estimators': [('model1', Pipeline(steps=[('scaler', StandardScaler()),
                ('estimator',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.1, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=5, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=200, n_jobs=0, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0,
     

### Representamos la importancia de cada feature

In [39]:
# Extraemos la feature importance del best model
feature_importance = best_model.named_steps['estimator'].feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
feature_names = X.columns
feature_names = feature_names[sorted_idx]
feature_importance = feature_importance[sorted_idx]
pos = np.arange(sorted_idx.shape[0]) + .5

# Representamos la importancia de las features
plt.figure(figsize=(12, 12))
plt.barh(pos, feature_importance, align='center')
plt.yticks(pos, feature_names)
plt.xlabel('Importancia relativa')
plt.title('Importancia de las features')
plt.show()


AttributeError: 'VotingRegressor' object has no attribute 'feature_importances_'

### Extraemos todas las métricas para este modelo en formato csv

In [ ]:
# Extraemos las métricas de evaluación para todos los modelos del pipeline en formato dataframe y lo pasamos a csv
cv_results = pd.DataFrame(gridsearch.cv_results_)


In [40]:
# Extraemos todas las métricas de evaluación del modelo en formato dataframe y luego lo pasamos a csv
metricas_votingregressor = pd.DataFrame({'MSE' : [mse], 'RMSE' : [rmse], 'MAE' : [mae], 'R2' : [r2]})


### Guardamos el mejor modelo en un archivo .pickle
#### (Pendiente de hacer función)

In [42]:
nombre_archivo = '../output/models/primer_procesado_cuarto_modelo_ensemble.pickle'
modelo = ensemble
import pickle
try:
    with open(nombre_archivo, 'wb') as archivo:
        pickle.dump(modelo, archivo)
    print(f"El modelo se ha guardado exitosamente en {nombre_archivo}.")
except IOError:
    print("Error: No se pudo guardar el modelo. Permiso denegado.")

El modelo se ha guardado exitosamente en ../output/models/primer_procesado_cuarto_modelo_ensemble.pickle.


In [44]:
def csv_datos(nombre_archivo, data):
    '''Guarda el dataframe data en un archivo csv con nombre nombre_archivo
    en la carpeta data/processed'''
    ruta_archivo = '../output/reports'
    data.to_csv(ruta_archivo + "/" + nombre_archivo, sep='\t', index=False)

In [45]:
# Guardamos los datos con las métricas en un csv
ruta_archivo = './output/reports/'
csv_datos("metricas_primer_procesado_cuarto_modelo_xgboost.csv", metricas_votingregressor)